# Create LoKI Model
Before training, you should first convert the original model into a model using LoKI Linear.


In [ ]:
from src.loki import create_loki_model

from src.loki import LoKILlamaForCausalLM
from src.loki import LoKILlamaConfig

from src.loki import LoKIQwen2ForCausalLM
from src.loki import LoKIQwen2Config

# create_loki_model(
#     loki_model_class=LoKILlamaForCausalLM,
#     loki_config_cls=LoKILlamaConfig,
#     target_pos_path="./kva_result/Llama-3.1-8B-Instruct/10.json",
#     save_dir="./models/loki",
#     model_name="meta-llama/Llama-3.1-8B-Instruct",
# )

create_loki_model(
    loki_model_class=LoKIQwen2ForCausalLM,
    loki_config_cls=LoKIQwen2Config,
    target_pos_path="./kva_result/Qwen2.5-0.5B-Instruct/10.json",
    save_dir="./models/loki",
    model_name= "Qwen/Qwen2.5-0.5B-Instruct"
)


# Check LoKI Model
You can test the converted LoKI model here.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "./models/loki"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True,
)
print(model)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

# Convert LoKI model to original model
After the training is completed, use the restore_roki_madel function to convert the LoKI model into a original model. In this way, you can naturally use general methods to call the trained model.


In [ ]:
from src.loki import restore_loki_model

restore_loki_model(
    model_path="./models/loki",
    target_pos_path="./kva_result/Qwen2.5-0.5B-Instruct/10.json",
    output_path="./models/output",
    model_name="Qwen/Qwen2.5-0.5B-Instruct",
)

# Merge LoRA adapter and convert the merged model to original model
If LoRA is used for training, LoRA adapter needs to be merged into the LoKI model before conversion.

In [ ]:
import torch
from transformers import AutoModelForCausalLM
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained("./models/loki", torch_dtype=torch.bfloat16, trust_remote_code=True,)
print(base_model)
lora_model = PeftModel.from_pretrained(base_model, "./models/lora_adapter")
merged_model = lora_model.merge_and_unload()
merged_model.save_pretrained("./models/merged")

restore_loki_model(
    model_path="",
    target_pos_path="./kva_result/Llama-3.1-8B-Instruct/10.json",
    output_path="./models/output",
    model_name="meta-llama/Llama-3.1-8B-Instruct",
)

# Suppress specified node output
The implementation code for the ablation section in the paper.

In [ ]:
from src.loki import set_zero_weights

set_zero_weights(
    target_pos_path="./kva_result/Llama-3.1-8B-Instruct/10.json",
    output_path="./models/suppressed",
    model_name="meta-llama/Llama-3.1-8B-Instruct"
)